# Test RTS/CTS

In [1]:
%discover
%connect nrf52
%rsync
%softreset -q

Hostname             URL                                      UID
esp32_cop            serial:///dev/cu.usbserial-014352DD      30:ae:a4:1a:27:28
nrf52                serial:///dev/cu.usbmodem1413401         c7:9d:75:c8:7a:14:1d:b6
Connected to nrf52 @ serial:///dev/cu.usbmodem1413401
with projects [secrets, airlift/libs/client]
UPDATE  lib/urpc/client.py


In [1]:
%connect nrf52

import board, busio, digitalio, io, time

with digitalio.DigitalInOut(board.MOSI) as cts, \
     digitalio.DigitalInOut(board.MISO) as rts:              
        print("cts", cts.value)
        print("rts", rts.value)
        

Connected to nrf52 @ serial:///dev/cu.usbmodem1413401
with projects [secrets, airlift/libs/client]
cts False
rts False


## Open

In [1]:
%connect nrf52
%softreset -q

import board, busio, io, time

uart = busio.UART(rx=board.TX, tx=board.RX, cts=board.MOSI, rts=board.MISO, baudrate=1_000_000)

Connected to nrf52 @ serial:///dev/cu.usbmodem1413401
with projects [airlift]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



* TX goes high, as expected
* all others stay low, as expected

## Loopback

In [1]:
%connect nrf52

import board, busio, io, time

try:
    uart.deinit()
except:
    pass

uart = busio.UART(rx=board.TX, tx=board.RX, \
                  baudrate=1_000_000, cts=board.MOSI, rts=board.MISO, \
                  receiver_buffer_size=8)

uart.write(b'abcdefghijklmnop')
for i in range(5):
    iw = uart.in_waiting
    res = uart.read(iw)
    print("[{:3}] {}".format(iw, res))
    time.sleep(0.1)

Connected to nrf52 @ serial:///dev/cu.usbmodem1413401
with projects [airlift]
[  8] b'abcdefgh'
[  6] b'jklmno'
[  0] b''
[  0] b''
[  0] b''


In [1]:
uart.write(b'123456789012')

12


* pulling CTS high stops transmission
* low/open -> transmission works

In [1]:
uart.write(b'START ')
if False:
    uart.write(b'abcdefghijklmnop1234')
    uart.write(b'abcdefghijklmnop1234')
    uart.write(b'abcdefghijklmnop1234')
    uart.write(b'abcdefghijklmnop1234')
    uart.write(b'abcdefghijklmnop1234')
    uart.write(b' END')


Interrupted

In [1]:
for i in range(5):
    iw = uart.in_waiting
    res = uart.read(iw)
    print("[{:3}] {}".format(iw, res))
    time.sleep(0.1)

[  8] b'12345678'
[  4] b'9012'
[  0] b''
[  0] b''
[  0] b''


## Echo

In [1]:
%connect nrf52

import board, busio, io, time

try:
    uart.deinit()
except:
    pass

uart = busio.UART(rx=board.TX, tx=board.RX, cts=board.MOSI, rts=board.MISO, \
                  baudrate=1_000_000, \
                  receiver_buffer_size=256)

# uart.write(b'abcdefghijklmnop')
while True:
    iw = uart.in_waiting
    if iw:
        res = uart.read(iw)
        print("[{:3}] {}".format(iw, res))
        uart.write(res)
    else:
        time.sleep(0.1)

Connected to nrf52 @ serial:///dev/cu.usbmodem1413401
with projects [secrets, airlift/libs/client]
[256] b'0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRST'
[256] b'UVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0'
[188] b'123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwx0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ'


Interrupted

## Client

In [1]:
%connect nrf52

import msgpack

try:
    _stream.deinit()
except:
    pass


class RPCError(Exception):
    pass


def import_(module: str):
    # import module on remote and return a reference
    return _send(("im", module))

def registry_():
    # remote object registry
    return _send(("_r", ))


def _obj_handler(obj):
    # convert _Proxy to its ext_type (in _send)
    if isinstance(obj, _Proxy):
        return obj.ext_type
    # should never happen
    raise ValueError("no handler for {}".format(obj))

    
def _ext_handler(code, data):
    # convert ExtType to _Proxy, etc
    if code == 1:
        return _Proxy(msgpack.ExtType(code, data))
    if code == 2:
        # error on remote
        # TODO: re-rise remote's error for tighter integration rather than wrapping in RPCError???
        raise RPCError(data.decode())
    if code == 3:
        # print on remote forwarded to host
        print(data.decode())
        # the actual result follows
        return msgpack.unpack(_stream, ext_hook=_ext_handler, use_list=False)
    raise ValueError("uRPC client received unknown ExtType({}, {})".format(code, data))


def _send(req):
    # clear rx buffer, required in case of comm error 
    # (e.g. too long response in last request)
    while _stream.in_waiting:
        iw = _stream.in_waiting
        _stream.read(_stream.in_waiting)
        print("_send: clear rx buffer: discarding {} bytes".format(iw))
    # send request
    msgpack.pack(req, _stream, default=_obj_handler)
    # timeout?
    while _stream.in_waiting == 0:
        pass
    res = msgpack.unpack(_stream, ext_hook=_ext_handler, use_list=False)
    return res


class _Proxy():
    
    def __init__(self, ext_type: msgpack.ExtType):
        # _Proxy is a reference (msgpack.ExtType) to an object on the remote
        self._ext_type = ext_type
        
    def __getattr__(self, name: str):
        # Note: 
        # We return an accessor function rather than a reference to the 
        # actual object (method, property) on the server. 
        # This avoids an extra rpc call but has the disadvantage that
        # properties cannot accessed the standard way.
        # (_Proxy.x just returns the accessor method, not the property value).
        def method(*args, **kwargs):
            return _send(("cm", self._ext_type, name, args, kwargs))       
        return method
    
    def get(self, name: str):
        # get object property
        return _send(("gp", self._ext_type, name))
         
    def set(self, name: str, value):
        # set object property
        return _send(("sp", self._ext_type, name, value))
    
    def __del__(self):
        # reclaim object on remote
        # Note: this could be automated with a finaliser (see FinaliserProxy)
        # Challenges:
        # 1) called from within gc - no allocation (calling _send is not permitted)
        # 2) in the present implementation the server uses the object id (address)
        #    for identification. Repeated execution of e.g. gc_ = import('gc') always
        #    returns the same reference but since on the client the "new" gc_ is a 
        #    different object it gets collected. If this function is called, the
        #    object is also deleted from the registry on the server.
        #    Workaround: replace the id with something that changes on each object
        #    creation, e.g. a counter. Not sure it works, what if functions a() and b()
        #    return references to the same object? The registry gets cluttered with all these refs?
        # --> Let's see if manual memory management on the server is really onerous.
        return _send(("_d", self._ext_type))
    
    def __str__(self):
        return _send(("_s", self._ext_type))
    
    
def start(stream=None):
    global _stream
    if not stream:
        # default is Particle Argon
        import board, busio
        stream = busio.UART(rx=board.TX, tx=board.RX, cts=board.MOSI, rts=board.MISO, \
                          baudrate=1_000_000, timeout=1, receiver_buffer_size=4096)
    _stream = stream
        
# start on import
start()

Connected to nrf52 @ serial:///dev/cu.usbmodem1413401
with projects [secrets, airlift/libs/client]


# URPC Client - Import from library

In [1]:
%connect nrf52

from urpc.client import *

Connected to nrf52 @ serial:///dev/cu.usbmodem1413401
with projects [secrets, airlift/libs/client]


In [1]:
# quick test
sys_ = import_('sys')
print(sys_.get('platform'))

esp32


In [1]:
gc_ = import_('gc')

for i in range(3):
    demo = import_('urpc_demo')
    obj = demo.Demo("obj # {:3}".format(i))
    print("[{:3}] mem_free".format(i), gc_.mem_free())
    # delete on server
    obj.__del__()
    del obj
    try:
        print("    obj =", obj)
    except NameError:
        pass

[  0] mem_free 90304
[  1] mem_free 89120
[  2] mem_free 87936


In [1]:
demo = import_('urpc_demo')
print("demo.a", demo.get('a'))
print("demo.upper", demo.upper("this is upper case!"))
obj = demo.Demo("my demo object")
print("obj =", obj)
print("obj.add:", obj.add(3,9))
print("obj.arg_demo:", obj.arg_demo('this is arg1', 3.1415, kw2='custom kw2'))
print("obj.desc:", obj.get('desc'))
obj.set('desc', 'new value')
print("obj.desc:", obj.get('desc'))
print("obj.new_attr:", obj.get('desc'))
obj.set('new_attr', 'newly created attribute')
print("obj.new_attr:", obj.get('new_attr'))
print("obj.dir:", obj.dir_demo())
print("obj.echo", obj.echo(12345))
print("obj.echo", obj.echo({'abc': 3.1415, 'xyz': 'value of xyz'}))

try:
    obj.foo(7)
except RPCError as e:
    print("\nRPCError:\n{}".format(e))
finally:
    # release object on server
    obj.__del__()
    
print("Registry:")
for k,v in registry_().items():
    print("  {} {}".format(k, v))

demo.a 5
demo.upper THIS IS UPPER CASE!
obj = <Demo id=1073729488 desc='my demo object'>
obj.add: 3 + 9 = 12
obj.arg_demo: arg1=this is arg1, arg2=3.1415, kw1=kw1 default, kw2=custom kw2
obj.desc: my demo object
obj.desc: new value
obj.new_attr: new value
obj.new_attr: newly created attribute
obj.dir: ('__class__', '__init__', '__module__', '__qualname__', '__str__', '__dict__', 'add', 'description', '_x', 'desc', 'arg_demo', 'dir_demo', 'echo', 'new_attr')
obj.echo 12345
obj.echo {'xyz': 'value of xyz', 'abc': 3.1415}

RPCError:
Traceback (most recent call last):
  File "/lib/urpc/server.py", line 81, in serve
  File "/lib/urpc/server.py", line 17, in cm
AttributeError: 'Demo' object has no attribute 'foo'

Registry:
  1073640432 <module 'urpc_demo' from '/lib/urpc_demo.py'>
  1061202660 <module 'gc'>
  1073657888 <Demo id=1073657888 desc='my demo object'>


In [1]:
import os, gc

N = 4000
data = bytes(os.urandom(N))

gc_  = import_('gc')
demo = import_('urpc_demo')
obj  = demo.Demo("my demo object")

for i in range(30):
    print("mem_free: host: {:6}  remote: {:6}".format(gc.mem_free(), gc_.mem_free()))
    assert data == obj.echo(data)
    
print("Match!")

mem_free: host: 133456  remote:  83568
mem_free: host: 129088  remote:  78992
mem_free: host: 124720  remote:  74416
mem_free: host: 120352  remote:  69840
mem_free: host: 115984  remote:  65264
mem_free: host: 111616  remote:  60688
mem_free: host: 107248  remote:  56112
mem_free: host: 102880  remote:  51536
mem_free: host:  98512  remote:  46960
mem_free: host:  94144  remote:  42384
mem_free: host:  89776  remote:  37808
mem_free: host:  85408  remote:  33232
mem_free: host:  81040  remote:  28656
mem_free: host:  76672  remote:  24080
mem_free: host:  72304  remote:  19504
mem_free: host:  67936  remote:  14928
mem_free: host:  63568  remote:  10352
mem_free: host:  59200  remote:   5776
mem_free: host:  54832  remote:   1200
mem_free: host:  50464  remote:  92240
mem_free: host:  46096  remote:  87664
mem_free: host:  41728  remote:  83088
mem_free: host:  37360  remote:  78512
mem_free: host:  32992  remote:  73936
mem_free: host:  28624  remote:  69360
mem_free: host:  24256  r